# Synthetic Data

Use the scraped data to produce similar sequences of moves that do not produce a solved cube. 

In [1]:
from transformers import PreTrainedTokenizerFast
from utils import Reconstruction
from tqdm import tqdm

import json
import random

with open("../solves.json", "r") as file:
    data = json.load(file)

tokenizer = PreTrainedTokenizerFast.from_pretrained("../rubiks-tokenizer")
N_UNSOLVED_SAMPLES = 5 * len(data)
P_CHANGE_MOVE = 0.1

/Users/henrywilliams/Documents/programming/python/ai/rubiks/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_samples = []
loop = tqdm("Generating new samples", total=N_UNSOLVED_SAMPLES)
attempts = 0
while len(new_samples) < N_UNSOLVED_SAMPLES:
    attempts += 1
    sample = random.choice(data)
    tokenized_solution = tokenizer(" ".join(sample["solve"]))
    new_solution = []

    for i, token in enumerate(tokenized_solution["input_ids"]):
        if token in [0, 1]:
            continue

        if random.random() <= P_CHANGE_MOVE:
            new_solution.append(random.randint(4, 69))
        else:
            new_solution.append(token)

    new_solve = tokenizer.decode(new_solution)
    tmp_r = Reconstruction("", " ".join(sample["scramble"]), new_solve)

    if not tmp_r.is_valid():
        new_samples.append(tmp_r.to_dict())
        loop.update()
        attempts = 0
    loop.set_postfix_str(f"Attempts: {attempts}")

100%|██████████| 43580/43580 [04:31<00:00, 163.12it/s, Attempts: 0]

In [4]:
from datasets import Dataset


def dataset_generator():
    for label, collection in enumerate([new_samples, data]):
        for sample in collection:
            yield {
                "scramble": sample["scramble"],
                "solve": sample["solve"],
                "is_solved": label,
            }


dataset = Dataset.from_generator(dataset_generator)
dataset = dataset.train_test_split(test_size=0.2)
dataset.save_to_disk("../rubiks-is-solved-dataset")

Saving the dataset (1/1 shards): 100%|██████████| 10460/10460 [00:00<00:00, 309551.47 examples/s]


In [ ]:
sample = random.choice(data)

In [ ]:
from utils import MoveSequence, cube_from_scramble
from rubik.solve import Solver

scramble = MoveSequence(" ".join(sample["scramble"]))
c = cube_from_scramble(str(scramble))
print(c)
solver = Solver(c)
solver.solve()
print("Solve: ", " ".join(solver.moves))
print("Generated solve: ", str(MoveSequence(" ".join(sample["solve"]))))

GYY
    ROB
    RBW
BBG WOY RGO BWR
RYO WWO GGY BBG
WWY RRG OOW ORO
    BWB
    GRY
    YYG

Solve:  B L L U B B B Ui R R Z Li Li E L Ei Li Ui B B B U Ei R E Ri Zi B B B B D Bi Di Z B B B D Bi Di Z B D Bi Di 
Z D B Di B B B Ri B B R Bi Bi D Bi Di Z B B L Bi Li Bi Di B D Z B B B B L Bi Li Bi Di B D Z B B B Bi Di B D B L Bi 
Li Z Bi Di B D B L Bi Li Z X X Xi Xi X X F Li Fi L D F Di Li F L F Li Fi L D F Di Li F L F F F Li Fi L D F Di Li F 
L F Xi Xi X X F Ri Fi R Fi Ri F F R F F F F F Xi Xi X X Ri S Ri Ri S S Ri Fi Fi R Si Si Ri Ri Si R Fi Fi Z Z Z Z Z 
Ri S Ri Ri S S Ri Fi Fi R Si Si Ri Ri Si R Fi Fi Zi Xi Xi

Generated solve:  Zi Y U R Mi Di L L Fi U D R Ui Ri U U Yi Ri Ui R R Ui Ui Ri U R Ui Ri Ui Yi R Ui Ui Ri U R Ui Ri 
Li U L U Li Ui L Ui Ui Li U L U Ri Ri M M Di R Mi U Ri M D R R Mi Mi Ui Ri M Ui R Mi Mi Mi Ui M U U Mi Ui Mi Mi U U